# Подключение гугл диска и загрузка библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# %cd /content/drive/My Drive/

In [ ]:
import numpy as np
import cv2
import pickle as pkl
import math
import torch
import os
import matplotlib.pyplot as plt
import requests
import tensorflow as tf
import random
import glob
from tensorflow.keras.preprocessing.image import load_img, array_to_img, ImageDataGenerator, img_to_array
from glob import glob
from PIL import Image

# Предобработка фоток

In [ ]:
# # В папке images1 - изначальные фотки сорняков
# # В папке labels1 - размеченные фотки сорняков
# def scale_resize_image(image):
#     image = tf.image.convert_image_dtype(image, tf.float32) # equivalent to dividing image pixels by 255
#     image = tf.image.resize(image, (640, 640)) # Resizing the image to 640x640 dimention
#     return image


# for pth in glob('/content/drive/MyDrive/осот/*.jpg'):
#     with Image.open(pth) as im:
#         new = scale_resize_image(im)._numpy()
#         tf.keras.preprocessing.image.save_img(pth, new)

In [ ]:
%cd /content/drive/MyDrive/финал/augmentation
!unzip /content/drive/MyDrive/финал/augmentation/augmentat.zip

/content/drive/MyDrive/финал/augmentation
Archive:  /content/drive/MyDrive/финал/augmentation/augmentat.zip
   creating: train/images/
  inflating: train/images/-_-1_jpg.rf.95715c7480b376c793496dc2c32c9cef.jpg  
  inflating: train/images/-_-10_jpg.rf.2e90cae178387db23bd105c971d92923.jpg  
  inflating: train/images/-_-11_jpg.rf.7570116441de313ef26253b37ebfe5ad.jpg  
  inflating: train/images/-_-12_jpg.rf.95f6eb37d4e1061addc8c4099d4a2656.jpg  
  inflating: train/images/-_-13_jpg.rf.4c079f8f560b28c26696c224d7ef67ab.jpg  
  inflating: train/images/-_-14_jpg.rf.b4f370995738b9161fb73f889f2ca760.jpg  
  inflating: train/images/-_-15_jpg.rf.014335bf1070eea0ff5cb096d87a75db.jpg  
  inflating: train/images/-_-16_jpg.rf.35d551b32b2d52028b83e7e4f2e17b6b.jpg  
  inflating: train/images/-_-17_jpg.rf.15ac5fc49aba95f33b83c5d65ff8e72f.jpg  
  inflating: train/images/-_-18_jpg.rf.f1aced9e0a6ad9c85f98f6410f08d774.jpg  
  inflating: train/images/-_-19_jpg.rf.91b253e651bd5fc745e039b8931a2110.jpg  
  inflat

# Функции и класс (для переворота картинок, разметки)

In [ ]:
def get_corners(bboxes):

    """Get corners of bounding boxes

    Parameters
    ----------

    bboxes: numpy.ndarray
        Numpy array containing bounding boxes of shape `N X 4` where N is the
        number of bounding boxes and the bounding boxes are represented in the
        format `x1 y1 x2 y2`

    returns
    -------

    numpy.ndarray
        Numpy array of shape `N x 8` containing N bounding boxes each described by their
        corner co-ordinates `x1 y1 x2 y2 x3 y3 x4 y4`

    """
    width = (bboxes[:,2] - bboxes[:,0]).reshape(-1,1)
    height = (bboxes[:,3] - bboxes[:,1]).reshape(-1,1)

    x1 = bboxes[:,0].reshape(-1,1)
    y1 = bboxes[:,1].reshape(-1,1)

    x2 = x1 + width
    y2 = y1

    x3 = x1
    y3 = y1 + height

    x4 = bboxes[:,2].reshape(-1,1)
    y4 = bboxes[:,3].reshape(-1,1)

    corners = np.hstack((x1,y1,x2,y2,x3,y3,x4,y4))

    return corners

In [ ]:
def rotate_im(image, angle):
    """Rotate the image.

    Rotate the image such that the rotated image is enclosed inside the tightest
    rectangle. The area not occupied by the pixels of the original image is colored
    black.

    Parameters
    ----------

    image : numpy.ndarray
        numpy image

    angle : float
        angle by which the image is to be rotated

    Returns
    -------

    numpy.ndarray
        Rotated Image

    """
    # grab the dimensions of the image and then determine the
    # centre
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    image = cv2.warpAffine(image, M, (nW, nH))

    # image = cv2.resize(image, (w,h))

# ...
    # image = img_to_array(image)
    # IMD = ImageDataGenerator()
    # return IMD.apply_transform(image, transform_parameters={'theta': angle})
# ...

    return image

In [ ]:
def rotate_box(corners,angle,  cx, cy, h, w):

    """Rotate the bounding box.


    Parameters
    ----------

    corners : numpy.ndarray
        Numpy array of shape `N x 8` containing N bounding boxes each described by their
        corner co-ordinates `x1 y1 x2 y2 x3 y3 x4 y4`

    angle : float
        angle by which the image is to be rotated

    cx : int
        x coordinate of the center of image (about which the box will be rotated)

    cy : int
        y coordinate of the center of image (about which the box will be rotated)

    h : int
        height of the image

    w : int
        width of the image

    Returns
    -------

    numpy.ndarray
        Numpy array of shape `N x 8` containing N rotated bounding boxes each described by their
        corner co-ordinates `x1 y1 x2 y2 x3 y3 x4 y4`
    """

    corners = corners.reshape(-1,2)
    corners = np.hstack((corners, np.ones((corners.shape[0],1), dtype = type(corners[0][0]))))

    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)


    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cx
    M[1, 2] += (nH / 2) - cy
    # Prepare the vector to be transformed
    calculated = np.dot(M,corners.T).T

    calculated = calculated.reshape(-1,8)

    return calculated

In [ ]:
def get_enclosing_box(corners):
    """Get an enclosing box for ratated corners of a bounding box

    Parameters
    ----------

    corners : numpy.ndarray
        Numpy array of shape `N x 8` containing N bounding boxes each described by their
        corner co-ordinates `x1 y1 x2 y2 x3 y3 x4 y4`

    Returns
    -------

    numpy.ndarray
        Numpy array containing enclosing bounding boxes of shape `N X 4` where N is the
        number of bounding boxes and the bounding boxes are represented in the
        format `x1 y1 x2 y2`

    """
    x_ = corners[:,[0,2,4,6]]
    y_ = corners[:,[1,3,5,7]]

    xmin = np.min(x_,1).reshape(-1,1)
    ymin = np.min(y_,1).reshape(-1,1)
    xmax = np.max(x_,1).reshape(-1,1)
    ymax = np.max(y_,1).reshape(-1,1)

    final = np.hstack((xmin, ymin, xmax, ymax,corners[:,8:]))

    return final

In [ ]:
def bbox_area(bbox):
    return (bbox[:,2] - bbox[:,0])*(bbox[:,3] - bbox[:,1])

In [ ]:
def clip_box(bbox, clip_box, alpha):
    """Clip the bounding boxes to the borders of an image

    Parameters
    ----------

    bbox: numpy.ndarray
        Numpy array containing bounding boxes of shape `N X 4` where N is the
        number of bounding boxes and the bounding boxes are represented in the
        format `x1 y1 x2 y2`

    clip_box: numpy.ndarray
        An array of shape (4,) specifying the diagonal co-ordinates of the image
        The coordinates are represented in the format `x1 y1 x2 y2`

    alpha: float
        If the fraction of a bounding box left in the image after being clipped is
        less than `alpha` the bounding box is dropped.

    Returns
    -------

    numpy.ndarray
        Numpy array containing **clipped** bounding boxes of shape `N X 4` where N is the
        number of bounding boxes left are being clipped and the bounding boxes are represented in the
        format `x1 y1 x2 y2`

    """
    ar_ = (bbox_area(bbox))
    x_min = np.maximum(bbox[:,0], clip_box[0]).reshape(-1,1)
    y_min = np.maximum(bbox[:,1], clip_box[1]).reshape(-1,1)
    x_max = np.minimum(bbox[:,2], clip_box[2]).reshape(-1,1)
    y_max = np.minimum(bbox[:,3], clip_box[3]).reshape(-1,1)

    bbox = np.hstack((x_min, y_min, x_max, y_max, bbox[:,4:]))

    delta_area = ((ar_ - bbox_area(bbox))/ar_)

    mask = (delta_area < (1 - alpha)).astype(int)

    bbox = bbox[mask == 1,:]


    return bbox

In [ ]:
def draw_rect(im, cords, color = None):
    """Draw the rectangle on the image

    Parameters
    ----------

    im : numpy.ndarray
        numpy image

    cords: numpy.ndarray
        Numpy array containing bounding boxes of shape `N X 4` where N is the
        number of bounding boxes and the bounding boxes are represented in the
        format `x1 y1 x2 y2`

    Returns
    -------

    numpy.ndarray
        numpy image with bounding boxes drawn on it

    """

    im = im.copy()

    cords = cords[:,:4]
    cords = cords.reshape(-1,4)
    if not color:
        color = [255,255,255]
    for cord in cords:

        pt1, pt2 = (cord[0], cord[1]) , (cord[2], cord[3])

        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])

        im = cv2.rectangle(im.copy(), pt1, pt2, color, int(max(im.shape[:2])/200))
    return im

In [ ]:
class Rotate(object):
    """Rotates an image


    Bounding boxes which have an area of less than 25% in the remaining in the
    transformed image is dropped. The resolution is maintained, and the remaining
    area if any is filled by black color.

    Parameters
    ----------
    angle: float
        The angle by which the image is to be rotated


    Returns
    -------

    numpy.ndaaray
        Rotated image in the numpy format of shape `HxWxC`

    numpy.ndarray
        Tranformed bounding box co-ordinates of the format `n x 4` where n is
        number of bounding boxes and 4 represents `x1,y1,x2,y2` of the box

    """

    def __init__(self, angle):
        self.angle = angle


    def __call__(self, img, bboxes):
        """
        Args:
            img (PIL Image): Image to be flipped.

        Returns:
            PIL Image: Randomly flipped image.


        """

        angle = self.angle

        w,h = img.shape[1], img.shape[0]
        cx, cy = w//2, h//2

        corners = get_corners(bboxes)

        corners = np.hstack((corners, bboxes[:,4:]))

        img = rotate_im(img, angle)
        corners[:,:8] = rotate_box(corners[:,:8], angle, cx, cy, h, w)




        new_bbox = get_enclosing_box(corners)


        scale_factor_x = img.shape[1] / w

        scale_factor_y = img.shape[0] / h

        img = cv2.resize(img, (w,h))

        new_bbox[:,:4] /= [scale_factor_x, scale_factor_y, scale_factor_x, scale_factor_y]


        bboxes  = new_bbox

        bboxes = clip_box(bboxes, [0,0,w, h], 0.25)

        return img, bboxes

In [ ]:
def check_for_class(pth):
    with open(f'{pth[:-3]}txt'.replace('images', 'labels')) as f:
        contents = f.readlines()
        return contents[0][0]

In [ ]:
def xywhn_to_xyxy():
    for pth in glob('/content/drive/MyDrive/финал/augmentation/train/labels/*.txt'):
        with open(pth, "rt") as f:
            x = np.array([[line.split()[0]] + [round(float(i) * 640) for i in line.split()[1:]] for line in f.readlines()])
            f.close()
        need = []
        for line in x:
            num_class = line[0]
            x = np.array(list(map(int, line[1:])))
            y = np.empty_like(x)
            w = 640
            h = 640
            dw = x[..., 2] / 2
            dh = x[..., 3] / 2
            y[..., 0] = x[..., 0] - dw
            y[..., 1] = x[..., 1] - dh
            y[..., 2] = x[..., 0] + dw
            y[..., 3] = x[..., 1] + dh
            need.append([int(num_class), *y.tolist()])
        with open(pth, "w") as f:
            f.write('\n'.join([' '.join([str(i) for i in line]) for line in need]))
            f.close()

In [ ]:
def xyxy_to_xywhn():
    for pth in glob('/content/drive/MyDrive/финал/augmentation/train/labels2/*.txt'):
        with open(pth, "rt") as f:
            x = np.array([[line.split()[0]] + [int(float(i)) for i in line.split()[1:]] for line in f.readlines()])
            f.close()
        need = []
        for line in x:
            num_class = line[0]
            x = np.array(list(map(int, line[1:])))
            y = np.empty_like(x)
            y[..., 0] = ((x[..., 0] + x[..., 2]) / 2)
            y[..., 1] = ((x[..., 1] + x[..., 3]) / 2)
            y[..., 2] = (x[..., 2] - x[..., 0])
            y[..., 3] = (x[..., 3] - x[..., 1])
            need.append([int(num_class), *(y / 640).tolist()])
        with open(pth, "w") as f:
            f.write('\n'.join([' '.join([str(i) for i in line]) for line in need]))
            f.close()

# Аугментация

In [ ]:
# xywhn_to_xyxy()

In [ ]:
paths = list(map(lambda x: '/content/drive/MyDrive/финал/augmentation/train/images/' + x, os.listdir('/content/drive/MyDrive/финал/augmentation/train/images')))

In [ ]:
paths[0]

'/content/drive/MyDrive/финал/augmentation/train/images-_-1_jpg.rf.95715c7480b376c793496dc2c32c9cef.jpg'

In [ ]:
# print(len(os.listdir('/content/drive/MyDriveimages/')))
print(len(paths))

101


In [ ]:
first = 0
second = 0
third = 0
fourth = 0
fifth = 0
sixth = 0
seventh = 0
eighth = 0
ninth = 0
for num_photo, pth in enumerate(paths):
    im = cv2.imread(pth)[:,:,::-1]

    num_class = int(check_for_class(pth))

    if num_class == 0:
        first += 1
        need = int(str(first)[:])
    elif num_class == 1:
        second += 1
        need = int(str(second)[:])
    elif num_class == 2:
        third += 1
        need = int(str(third)[:])
    elif num_class == 3:
        fourth += 1
        need = int(str(fourth)[:])
    elif num_class == 4:
        fifth += 1
        need = int(str(fifth)[:])
    elif num_class == 5:
        sixth += 1
        need = int(str(sixth)[:])
    elif num_class == 6:
        seventh += 1
        need = int(str(seventh)[:])
    elif num_class == 7:
        eighth += 1
        need = int(str(eighth)[:])
    elif num_class == 8:
        ninth += 1
        need = int(str(ninth)[:])
    in_coor  = ['class','x1','y1','x2','y2']
    out_coor = ['x1','y1','x2','y2','class']
    with open(f'{pth[:-3]}txt'.replace('images', 'labels')) as f:
        contents = f.readlines()
        f.close()
    data = list()
    for line in (contents):
        data.append([float(j) for j in line.split(' ')])
    data_arr  = np.asarray(data)
    in_order  = {j:i for i,j in enumerate(in_coor)}
    sort_in   = [in_order[k] for k in in_coor]
    sort_out  = [in_order[k] for k in out_coor]
    bboxes = data_arr[:,sort_out]

    for num_rotation, degree in enumerate(range(0, 360, 90)):
        rotation = Rotate(degree)
        new_img, new_bboxes = rotation(im, bboxes)

        # plotted_img = draw_rect(new_img.copy(), new_bboxes.copy())
        # plt.imshow(plotted_img)
        # plt.show()

        image = Image.fromarray(new_img, 'RGB')

        for labeled in range(6):
            with open(f'/content/drive/MyDrive/финал/augmentation/train/labels2/{num_class}_{need}_{num_rotation + 1}_{labeled}.txt', "w") as f:
                f.write('\n'.join([' '.join(list(map(str, [int(bbox[-1])] + [i for i in bbox[:-1]]))) for bbox in new_bboxes.tolist()]))
                f.close()

        tf.keras.preprocessing.image.save_img(f'/content/drive/MyDrive/финал/augmentation/train/images2/{num_class}_{need}_{num_rotation + 1}_{0}.jpg', image)

        grayscaled = tf.image.rgb_to_grayscale(image)
        tf.keras.preprocessing.image.save_img(f'/content/drive/MyDrive/финал/augmentation/train/images2/{num_class}_{need}_{num_rotation + 1}_{1}.jpg', grayscaled)

        blured = cv2.GaussianBlur(new_img, (5, 5), cv2.BORDER_DEFAULT)
        blured = Image.fromarray(blured, 'RGB')
        tf.keras.preprocessing.image.save_img(f'/content/drive/MyDrive/финал/augmentation/train/images2/{num_class}_{need}_{num_rotation + 1}_{2}.jpg', blured)

        gnoise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.1, dtype=tf.float32)
        image_type_converted = tf.image.convert_image_dtype(image, dtype=tf.float32, saturate=False)
        noisy_image = tf.add(image_type_converted, gnoise)
        tf.keras.preprocessing.image.save_img(f'/content/drive/MyDrive/финал/augmentation/train/images2/{num_class}_{need}_{num_rotation + 1}_{3}.jpg', noisy_image)

        brighted = tf.image.adjust_brightness(image, 0.25)
        tf.keras.preprocessing.image.save_img(f'/content/drive/MyDrive/финал/augmentation/train/images2/{num_class}_{need}_{num_rotation + 1}_{4}.jpg', brighted)

        saturated = tf.image.adjust_saturation(image, 4)
        tf.keras.preprocessing.image.save_img(f'/content/drive/MyDrive/финал/augmentation/train/images2/{num_class}_{need}_{num_rotation + 1}_{5}.jpg', saturated)

In [ ]:
xyxy_to_xywhn()

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/финал/augmentation/train/labels2')))

2424


In [ ]:
# В папке images2 - аугментированные фотки сорняков
# В папке labels2 - аугментированные разметки сорняков